## Connexion de Google Drive
Pour télécharger directement les fichiers sur votre Google Drive, exécutez ensuite cette cellule et suivez les instructions. La commande crée un lien qui ouvre sur une nouvelle page vous demandant l'autorisation d'accéder à votre Drive, puis vous donne un code à copier-coller ci-dessous et à activer avec Entrée:


In [ ]:
"""from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/MyDrive/journaux-xix-siecle/main"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Récupération de numéros sur Gallica

---






S'il n'existe pas encore, créez le dossier dans lequel vous voulez télécharger les images puis pointez vers lui (en modifiant les cellules ci-dessous, /My Drive/ est la racine de Google Drive).




In [ ]:
import urllib.request, urllib.error, urllib.parse
import re
import os
import json

# Initiating the function for retrieving the date id of gallica
def pressdate(year, month, day, rate, item):
    finallist = []
    monthbissex = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    monthunbissex = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    counting = 1
    curmonth = month - 1
    initialday = '%02d' % day
    initialmonth = curmonth + 1
    initialmonth = '%02d' % initialmonth
    initialresult = str(year) + str(initialmonth) + str(initialday)
    finallist.append(initialresult)
    if year % 4 == 0 and not year % 100 == 0:
        while counting < item:
            day += rate
            counting += 1
            if day > monthbissex[curmonth]:
                day = day - monthbissex[curmonth]
                curmonth += 1
            if curmonth == 12:
                year += 1
                curmonth = 0
            finalmonday = '%02d' % day
            finalmonth = curmonth + 1
            finalmonth = '%02d' % finalmonth
            finalresult = str(year) + str(finalmonth) + str(finalmonday)
            finallist.append(finalresult)
    else:
        while counting < item:
            day += rate
            counting += 1
            if day > monthunbissex[curmonth]:
                day = day - monthunbissex[curmonth]
                curmonth += 1
            if curmonth == 12:
                year += 1
                curmonth = 0
            finalmonday = '%02d' % day
            finalmonth = curmonth + 1
            finalmonth = '%02d' % finalmonth
            finalresult = str(year) + str(finalmonth) + str(finalmonday)
            finallist.append(finalresult)
    return finallist


def jpgpress(url, title="titre", year=1900, month=1, day=1, item=5, rate=1, firstpage=1, lastpage=1, resolution=5000, images_file_name="images"):
    firstdate = pressdate(year, month, day, rate, item)
    secondpage = firstpage + 1
    lastpage += 1
    listpage = range(secondpage, lastpage)

    

    extension = '.jpg'

    for date in firstdate:
        jpgfile = title + "_" + date + "_page_" + str(firstpage) + extension
        firsturl = url + date
        response = urllib.request.urlopen(firsturl)
        realurl = response.geturl()
        print(realurl)
        identifier = re.sub(r'https://gallica.bnf.fr/ark:', r'', realurl)
        identifier = re.sub(r'.item', r'', identifier)
        finalurl = 'https://gallica.bnf.fr/iiif/ark:' + identifier + '/f' + str(firstpage) + '/full/' + str(resolution) + '/0/native' + extension
        print(finalurl)
        try:
            urllib.request.urlopen(finalurl)
        except urllib.error.URLError as e:
            if hasattr(e, 'reason'):
                print('We failed to reach a server.')
                print('Reason: ', e.reason)
            elif hasattr(e, 'code'):
                print('The server couldn\'t fulfill the request.')
                print('Error code: ', e.code)
        else:
            urllib.request.urlretrieve(finalurl, images_file_name + '/' + jpgfile)
        if len(listpage) > 0:
            for page in listpage:
                jpgfile = title + "_" + date + "_page_" + str(page) + extension
                otherurl = 'https://gallica.bnf.fr/iiif/ark:' + identifier + '/f' + str(
                    page) + '/full/' + str(resolution) + '/0/native' + extension
                try:
                    urllib.request.urlopen(otherurl)
                except urllib.error.URLError as e:
                    if hasattr(e, 'reason'):
                        print('We failed to reach a server.')
                        print('Reason: ', e.reason)
                    elif hasattr(e, 'code'):
                        print('The server couldn\'t fulfill the request.')
                        print('Error code: ', e.code)
                else:
                    urllib.request.urlretrieve(otherurl, images_file_name + '/' + jpgfile)


Modifiez la ligne de la cellule ci-dessous en fonction des numéros du périodique que vous voulez télécharger, puis exécutez la cellule:

jpgpress(url="http://gallica.bnf.fr/ark:/12148/cb32817642h/date", title="moderniste", year=1889, month=5, day=25, item=4, rate=7, firstpage=1, lastpage=12)
La fonction comprend les commandes suivantes :

url: Pour récupérer les images d’une série de numéros de périodiques, on part de l’adresse du périodique avec les dates:http://gallicalabs.bnf.fr/ark:/12148/cb34427442r/date.

title: on choisit un titre qui sera indiqué dans le nom du fichier.

year, month, day: la date du premier numéro qu’on souhaite télécharger.

item: le nombre de fichiers qu’on veut récupérer.

rate: le nombre de jours entre chaque numéro – cela fonctionne bien pour les quotidiens (on indique 1), les hebdomadaires (on indique 7); en cas de problème, on peut inscrire 1 et préciser dans “item” le nombre total de jours dans la période qu’on veut télécharger (par exemple, pour un an on inscrira 365), cela permettra d’être certain de télécharger tous les numéros, même si cela produit des erreurs (les dates sans numéros).

firstpage et lastpage: la première et la dernière page de chaque numéro qu'on veut télécharger; ceci peut permettre par exemple de ne télécharger que les unes. Pour être certain de télécharger toutes les pages d'un périodique, il faut ici tenir compte de la pagination; on peut gonfler le nombre de pages au cas ou certains documents comportent plus de pages, les erreurs s’afficheront dans le shell avant que le programme ne continue avec le numéro suivant du périodique. Par exemple, pour un journal de 8 pages, il vaut mieux indiquer 12 pages pour être certain que des numéros plus longs seront entièrement téléchargés.

resolution: changer la résolution des images obtenues pour avoir une plus ou moins bonne qualité. Cela peut être un nombre comme 3000 ou 5000, pour avoir la qualité maximale nous pouvons mettre 'full' au lieu d'un nombre.

On peut aussi récupérer les images au format png ou tif en remplaçant la mention « jpg » par « png » ou « tif » à la fin de l’adresse et du format de fichier créé.

Les numéros de périodiques sont enregistrés sur votre Google Drive. Vous pouvez observer la réussite des téléchargements dans la barre de gauche, sous l'onglet "fichiers".

In [ ]:
with open('datas/journaux.json') as d:
  journauxDict = json.load(d)

In [ ]:
images_file_name = 'images'
try:
  os.makedirs('./' + images_file_name)
except: pass

%cd './' + images_file_name

[Errno 2] No such file or directory: './ + images_file_name'
/content/drive/MyDrive/journaux-xix-siecle/main


In [ ]:
#jpgpress(url="https://gallica.bnf.fr/ark:/12148/cb34520232c/date", title="L'Univers", year=1867, month=10, day=24, item=1, rate=1, firstpage=1, lastpage=12, resolution='full')
#jpgpress(url="https://gallica.bnf.fr/ark:/12148/cb32868136g/date", title="Le Siècle", year=1836, month=7, day=1, item=5, rate=1, firstpage=1, lastpage=12, resolution='full')

for key in journauxDict:
  journal = journauxDict[key]
  jpgpress(url=journal['url'], title=journal['title'], year=journal['year'], month=journal['month'], day=journal['day'], item=journal['item'], rate=journal['rate'], firstpage=journal['firstpage'], lastpage=journal['lastpage'], resolution=journal['resolution'], images_file_name=images_file_name)


https://gallica.bnf.fr/ark:/12148/bpt6k697462c.item
https://gallica.bnf.fr/iiif/ark:/12148/bpt6k697462c/f1/full/full/0/native.jpg
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
https://gallica.bnf.fr/ark:/12148/bpt6k7183151.item
https://gallica.bnf.fr/iiif/ark:/12148/bpt6k7183151/f1/full/full/0/native.jpg
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
We failed to reach a server.
Reason:  404
https://gallica.bnf.fr/ark:/12148/bpt6k718316d.item
https://gallica.